In [1]:
import pandas as pd
import numpy as np
import os

**Step 1. Read the QC report file for all TFs and filter out low quality TFs.**

You will need to download the QC table (named as "**human_factor_full_QC**") by yourself from CistromeDB: http://cistrome.org/db/#/bdown

In [4]:
qc = pd.read_csv("data/raw/cistromedb/human_factor_full_QC.txt",sep = "\t")

From http://cistrome.org/chilin/_downloads/instructions.pdf
1. FastQC is the sample’s median sequence quality scores. ChiLin calculates these scores using the
FastQC software[2]. **A good sequence quality score is ≥ 25.**
2. Original total reads is the sample’s raw total reads number.
3. Uniquely mapped reads is the number of reads with mapping quality above 1. First, ChiLin aligns
reads onto user-specified genomes. Then, it filters the SAM files. The **uniquely mapped RATIO** is the
uniquely mapped reads divided by the total reads. **A good uniquely mapped ratio is ≥ 60%.**
4. Unique locations of 4M reads is the number of genomic locations with one or more uniquely mapped
reads (unique locations) from sub-sampled 4M reads. Unique locations ratio unique locations number
divided by total number of uniquely mapped reads. ChiLin estimates NRF by dividing the number of
unique locations by 4M sampled uniquely mapped reads. If reads are less than 4M, then ChiLin uses
the total reads instead. ChiLin reports number of unique locations and the unique locations ratio. A
good unique locations of 4M reads should be ≥ 70%.
5. Locations with only 1 read from 4M reads number (ratio) is the number of locations with read
number equal to 1 (N1). The ratio is N1 divided by 4M reads unless the total reads is less than 4M, in
which case the total reads is used. A good score for this metric is > 70%.
6. PBC of 4M reads is N1 (see 5) divided by unique locations (see 4). **A good PBC score is ≥ 80%.**
7. Fragment size of 4M reads is in silico estimation of your size selection through maximum cross
correlation. The estimation should to be close to the size selected in your experiment.
8. Exon/DHS/Promoter ratio of 4M reads is the estimated ratio of reads falling in these regions (from
a 4M reads sub-sample). Exons regions are defined as the merged exons regions from the RefSeq
gene table. Promoter regions are defined as the RefSeq TSS +/- 2kb regions. Union DHS regions are
called from ENCODE II UW DNase-seq Hypersensitive regions. The IP group samples should have
higher reads ratios than the control group samples.
9. **FRiP of 4M non-chrM reads is used for evaluating the signal to noise ratio.** First, ChiLin removes
chrM reads from the total reads. Then ChiLin sub-samples 4M of these reads. Finally, it calculates
the ratio of the sub-sample which fall under the called peaks. **A good FRiP score is ≥ 1%.**
10. Replicates total peaks are the total peaks number called by MACS2 with fixed extension size and q
value cutoff. A good peaks number depends on your experiment.
11. Replicates 10 fold confident peaks are the number of peaks called by MACS2 where the fold change
is ≥ 10. **A good number is above 500**

Select samples with high quality, total number of selected TFs = 623

In [5]:
qc = qc.loc[(qc.FastQC >= 25) & 
 (qc.UniquelyMappedRatio >= 0.6) &
 (qc.PBC > 0.8) &
 (qc.FRiP > 0.01) &
 (qc.PeaksFoldChangeAbove10 > 500), ]

In [6]:
# Change SMAD2/3 -> SMAD23 so that we can use the TF name as file name
qc.loc[qc["Factor"] == "SMAD2/3","Factor"] = "SMAD23"

**Step2. Check number of cells, and number of peaks for each tissue**

You will need download the scATAC-seq datasets with the sample IDs shown in the following cell. These datasets can be downloaded from the HuBMap data portal: https://portal.hubmapconsortium.org/. Alternatively, you may perfrom batch download using commandline tool as described in https://software.docs.hubmapconsortium.org/clt/index.html. This script assumes you have downloaded the data into the folder **data/raw/atac/** and the subfolders in it were all named by sample ID. For example, for a sample with the ID "73471388c8fb65d21f964a1df408db1f", its ATAC-seq bed file (peaks.combined.bed) can be found inside data/raw/rna/73471388c8fb65d21f964a1df408db1f/.

In [4]:
from anndata import read_h5ad
import h5py

rootdir = "data/atac"

# Iterate over tissues, read in cell by bin matrix, and peaks files. Then count number of cells and peaks.
for tissue in ["Liver","Heart","Kidney (Left)","Large Intestine","Spleen","Lung (Right)"]:
    
    n_peaks = 0
    n_cells = 0
    
    # Read peaks and cell by bin matrix
    for Dir in os.listdir(rootdir + tissue):
        if os.path.exists("/".join([rootdir,tissue,Dir,"peaks.combined.bed"])) and os.path.exists("/".join([rootdir,tissue,Dir,"cell_by_gene.hdf5"])):
            print(Dir)
            peaks = pd.read_csv("/".join([rootdir,tissue,Dir,"peaks.combined.bed"]),
                            sep = "\t", header = None)
            n_peaks += peaks.shape[0]
            n_cells += h5py.File("/".join([rootdir, tissue, Dir, "cell_by_gene.hdf5"]),"r")['row_names'].shape[0]

    print("{} completed, {} cells, {} peaks".format(tissue, n_cells, n_peaks))

73471388c8fb65d21f964a1df408db1f
7e6657cc565a7d89a9041dece3816ffa
Liver completed, 50748 cells, 113726 peaks
bf0f0ca3a57f5d82141c27f98ec6eb40
01d94c78c858b9944b4bbdd5b273c2bd
225d7c6d4843b97ec4f6a9acc0f82dda
59e8b98ce6094e0b73c627edb5b00a09
921c37b95f73de863c95dd97c3393d1b
6e89c8eb27e1ef96bbbf4c48447dc4dc
53cc1f9c21907d8f7880eed6828ba8a1
fc5eb8a90d7caa1b8ea5d920b3d289c9
1324f228f23b7ec156c6c58a66bd58fd
4c390581222e1b00966bb7406f6f8073
29610875ec053f0efb6c493c8329c1b8
d4493657cde29702c5ed73932da5317c
Heart completed, 310661 cells, 270033 peaks
75d8ae60d6d2be8ba71ff5213940917d
8a3b11f9934baa372696c240fcc14876
9d534e3b706b1e88f4960077e651aa28
81c9ebd67ad019ef2589f883d31e25c9
f8de2f78bb01fbd41dd71cedd2b60750
751eb90f02674b9c77cfed31155cd855
d595473c117c90b357b1584f652fbcbc
ccdb215b6931b3989b3960adf3ac4657
717ec64b27b77e6ede9cb531b4edd786
5c469cc1e2efa28c64389e1ff1549207
73a351ecd690eb11ab6032445bcc7ae2
2c1f879670564dff2e541e239ca96344
88b6918eac299aa7a2bd0a8e8cf75236
0699ea08c9e0bfbed61bcf

**Step 2. Read the scATAC-seq peaks, and build scATAC-seq search tree for each tissue and each chromosome**

In [5]:
from intervaltree import IntervalTree,Interval
from collections import defaultdict
import re

rootdir = "data/atac/"
search_tree = defaultdict(dict)

# Iterate over tissues and chromosomes. For each tissue, buid interval trees.
for tissue in os.listdir(rootdir):
    
    # Skip pancreas
    if os.path.isdir(rootdir + tissue) and tissue != "Pancreas":
        peaks = []
        for Dir in os.listdir(rootdir + tissue):
            peaks.append(pd.read_csv("/".join([rootdir,tissue,Dir,"peaks.combined.bed"]),
                            sep = "\t")
                        )
            peaks[-1].columns = ["chrom","chromStart","chromEnd"]
        
        # Combine peaks from the same tissue
        peaks = pd.concat(peaks)
        peaks['chrom'] = 'chr' + peaks['chrom']

        # For each chromosome, build an interval tree for search.
        for chrom,df in peaks.groupby('chrom'):
            
            # Remove parenthesis and spaces in the tissue name
            tissue = re.sub('\s+',"_",tissue)
            tissue = re.sub('\(',"",tissue)
            tissue = re.sub('\)',"",tissue)
    
            search_tree[tissue][chrom] = IntervalTree(Interval(row.chromStart,row.chromEnd) for i,row in df.iterrows())
            search_tree[tissue][chrom].merge_overlaps()
        print("{} completed".format(tissue))

Liver completed
Lung_Right completed
Spleen completed
Kidney_Left completed
Kidney_Right completed
Large_Intestine completed
Heart completed


**Step 3. For each ChIP-seq peaks of TF, compute the overlapping regions with scATAC-seq, and map the peak to nearest genes.**
Need to install ChIPseeker R package before performing this step. Note that for T transcription factor, R will recognize character "T" as **True** value instead of character "T".

In [ ]:
# Compute overlapping weights
from joblib import Parallel, delayed
import subprocess
import pdb
import shutil

# Create tissue subfolder for saving TFs.
for tissue in search_tree.keys():
    
    # remove subfolder if it already exists.
    shutil.rmtree("data/processed/chip+atac/{}".format(tissue), ignore_errors=True)
    os.mkdir("data/processed/chip+atac/{}".format(tissue))

def search_overlap(tf,df):
    
    # Read all peak files for a TF
    peaks = []
    for idx,row in df.iterrows():
        peaks.append(
                        pd.read_csv("data/processed/cistromedb/" + str(row.DCid) + "_sort_peaks.narrowPeak.bed",
                        sep = "\t",
                        header = None)
                    )
        peaks[-1].columns = ["chrom",
                         "chromStart",
                         "chromEnd",
                         "name",
                         "score",
                         "strand",
                         "signalValue",
                         "pValue",
                         "qValue",
                         "peak"]
        peaks[-1]["tf"] =  np.repeat(row.Factor,peaks[-1].shape[0])
        peaks[-1]["DCid"] = np.repeat(row.DCid,peaks[-1].shape[0])
        
    peaks = pd.concat(peaks)

    # Search the overlapped between scATAC-seq peaks and ChIP-seq peaks for each TF in each tissue
    # each row of 'peaks' represents a peak in ChIP-seq data.
    # each 'peaks' table represents all peaks of a TF from ChIP-seq data.
    # 1. iterate over all tissues
    for tissue,tree in search_tree.items():
    
        # 2. iterate over all peaks of a TF
        res = []
        for idx,row in peaks.iterrows():
            chrom = row['chrom']
            start = row['chromStart']
            end = row['chromEnd']
            sigVal = row['signalValue']

            if chrom in tree:
                intervals = tree[chrom][start:end]
                intervals = sorted(list(intervals), key = lambda x: x[0])
                over_len = 0
                for interval in intervals:
                    over_len += min(end,interval[1]) - max(start,interval[0])
                res.append(over_len/(int(end) - int(start)))
            else:
                res.append(None)
        
        # After all peaks of a TF is done. All peaks of the TF is saved as one file, specific to each tissue.
        peaks["chip_atac_weight"] = res
        outname = "data/processed/chip+atac/{}/{}.csv".format(tissue,tf)
        peaks.to_csv(outname)

        # map TF peaks to nearest genes
        commands = [
            "module load singularity",
            "Rscript annotate_peaks.R {} {}".format(outname,outname),
        ]
        commands = ";".join(commands)
        subprocess.run(commands, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
    print("{} done;".format(tf))
    return(None)

res = Parallel(n_jobs=14,backend='multiprocessing')(delayed(search_overlap)(tf,df) 
                              for i,(tf,df) in enumerate(qc.groupby("Factor")))

APOBEC3B done;
ARID1A done;
AHR done;
AFF1 done;
ASXL1 done;
ASH2L done;
ARID2 done;
ASCL1 done;
ARRB1 done;
AFF4 done;
ARID3A done;
ATF2 done;
ATF1 done;
BARX2 done;
BATF3 done;
BACH2 done;
BCL11B done;
BACH1 done;
BATF done;
ATF4 done;
BCLAF1 done;
BIRA done;
BPTF done;
BRD1 done;
BCOR done;
BRCA1 done;
BCL6 done;
BMI1 done;
BHLHE40 done;
BRD3 done;
BRD2 done;
BCL3 done;
BRD9 done;
C17orf49 done;
C11orf30 done;
CBX8 done;
CBX5 done;
CBX1 done;
CBFB done;
CCNT2 done;
CDK6 done;
ATF3 done;
CDK7 done;
CBX3 done;
CDX2 done;
CHD1 done;
CEBPD done;
CHD8 done;
CLOCK done;
CEBPG done;
CDK9 done;
CHD2 done;
CHRM2 done;
CDK8 done;
CPSF3L done;
CEBPA done;
CREB3 done;
CREM done;
CNOT3 done;
CTBP1 done;
CREBBP done;
CTNNB1 done;
CTCFL done;
CUX1 done;
DCP1A done;
DDX21 done;
DDX20 done;
DAXX done;
DEAF1 done;
DIDO1 done;
DEK done;
DLX1 done;
DPF1 done;
DNMT3A done;
DRAP1 done;
E2F3 done;
DPF2 done;
E2F4 done;
E2F5 done;
DUX4 done;
E2F6 done;
CREB1 done;
E2F7 done;
E2F8 done;
E2F1 done;
EBNA3C do

**Step 4. For each tissue, read the results and generate the TF-gene activity matrix**

Get non-zero and non-na TF-gene pairs

In [57]:
import os
from collections import defaultdict

folder = "data/processed/chip+atac/"
os.chdir(folder)
tissues = os.listdir()
tf_gene = defaultdict(list)

for tissue in tissues:
    os.chdir(folder + tissue)
    print("================{}================".format(tissue))
    tf_files = os.listdir() 
    
    for idx,tf_file in enumerate(tf_files):
        df = pd.read_csv(tf_file)
        tf_gene[tissue].append(df.loc[~df["gene"].isna() & (df["chip_atac_weight"] != 0),["tf","gene","chip_atac_weight"]])
        print("{} done, {}/{} finished".format(tf_file, idx+1, len(tf_files)),end = '\r')
    tf_gene[tissue] = pd.concat(tf_gene[tissue])

================Kidney_Left================
================Heart================heded
================Lung_Right================
================Kidney_Right================
================Large_Intestine================
================Spleen================eded
================Liver================heded


Get mean of weights for each TF-gene pair

In [ ]:
tf_gene_mean = defaultdict(list)
for tissue,tab in tf_gene.items():
    print("================{}================".format(tissue))
    total_num = tab.loc[:,["tf","gene"]].drop_duplicates().shape[0]
    for idx,((tf,gene),df) in enumerate(tab.groupby(["tf","gene"])):
        tf_gene_mean[tissue].append({"tf":tf,
         "gene":gene,
         "chip_atac_weight":df["chip_atac_weight"].mean()})
        print("{}/{} done {}% finished".format(idx+1, total_num, np.round((idx+1)/total_num * 100,2)),end = '\r')
    tf_gene_mean[tissue] = pd.DataFrame(tf_gene_mean[tissue])

================Kidney_Left================
================Heart================
================Lung_Right================
================Kidney_Right================
================Large_Intestine================
================Spleen================
================Liver================


Get sum of weights for each TF-gene pair

In [59]:
tf_gene_sum = defaultdict(list)
for tissue,tab in tf_gene.items():
    tf_gene_sum[tissue] = tab.groupby(["tf","gene"], as_index = False)['chip_atac_weight'].sum()

Convert data frames to tf-gene activities matrices

In [68]:
tf_gene_mat_mean = dict()
for tissue,mat in tf_gene_mean.items():
    tf_gene_mat_mean[tissue] = mat.pivot(index = "tf", columns = "gene", values = "chip_atac_weight")

tf_gene_mat_sum = dict()
for tissue,mat in tf_gene_sum.items():
    tf_gene_mat_sum[tissue] = mat.pivot(index = "tf", columns = "gene", values = "chip_atac_weight")

Save tf-gene activities matrix

In [79]:
for tissue in tf_gene_mat_mean.keys():
    tf_gene_mat_mean[tissue].to_csv("data/processed/tf_activity/mean_tf/{}_tfGeneMat.csv".format(tissue))
    
for tissue in tf_gene_mat_sum.keys():
    tf_gene_mat_sum[tissue].to_csv("data/processed/tf_activity/sum_tf/{}_tfGeneMat.csv".format(tissue))